---
# Projectr : 하루시작 지하철 혼잡도 분석
### Description : 
- <a><span style = "color : #FFBE98">**오늘의 주요 키워드 및 뉴스 추천하기**</a>
    
### Author : Zen Den
### Date : 2024. 06. 11. (Tue) ~
### Detail : 
### Update: 
- 2024.06.11. (Tue) K.Zen : 
  <a href = "https://news.naver.com/main/ranking/popularDay.naver">
    <span style = "color : #F7CAC9">**NAVER 랭킹뉴스**</a>
에서 언론사별 5개씩 가져오기
  <span style = "color : #FFBE98">**(언론사명, Title, Link)**</span>
  <br><br>
- 2024.06.12. (Wed) K.Zen : <br><br>
  - Selenium Library를 이용한 크롤링 과정 중 ScreenShot 촬영을 백그라운드로 실행하기 <br>
  [참고자료]
    <a href = "https://co-de.tistory.com/21">
      <span style = "color : #F7CAC9">**[selenium]** 손쉽게 브라우저 자동 캡쳐 기능 만들기</a>

---
# Analyse News Articles by media outlet to recommend <br> the top keywords and news of the day

## Import Library Package

### Basic

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, warnings; warnings.filterwarnings('ignore')

### File System

In [2]:
import os; from pathlib import Path
from datetime import datetime

### Crawling

In [3]:
import requests # 인터넷에서 Data를 가져오기 위한 Library
from bs4 import BeautifulSoup # 웹 페이지 내용을 분석하기 위한 Library
import time # 대기 시간을 추가하기 위한 Library
import random # Random한 대기 시간을 만들기 위한 Library
from tqdm import tqdm # Crawling 진행 상황을 체크하기 위한 Module

from multiprocessing import Pool

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

### Bag of Words (BoW)

In [4]:
from konlpy.tag import Okt
import nltk # Natural Language Toolkit
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter

### Deep Learning

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification

# NAVER News - 언론사별 랭킹뉴스 Crawling

## 언론사, Title, Link 추출하기

In [6]:
# 뉴스 Crawling
def get_news_links_by_press (url) :
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
  response = requests.get(url, headers = headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  
  press_data = {}
  press_sections = soup.select('.rankingnews_box')
  
  for press_section in tqdm(press_sections, desc = "언론사별 뉴스 Crawling") :
    press_name = press_section.select_one('.rankingnews_name').get_text(strip = True)
    news_links = set()  # 중복 제거를 위한 set 사용
    for item in press_section.select('li a') :
      title = item.get_text(strip = True)
      link = item['href']
      if title and link and "동영상" not in title :  # Title이 존재하고 "동영상"이 포함되지 않은 경우에만 추가
        news_links.add((title, link))
    press_data[press_name] = list(news_links)[:5]  # 다시 list로 변환 후 상위 5개만 저장
    
    # 각 언론사별 뉴스 Crawling 후 대기 시간 추가
    time.sleep(random.uniform(0.5, 2.0))
  
  return press_data

In [7]:
base_url = 'https://news.naver.com/main/ranking/popularDay.naver'
press_news_data = get_news_links_by_press(base_url)

# 뉴스 DataFrame 생성
news_list = []
for press_name, news_data in press_news_data.items() :
  for title, link in news_data :
    news_list.append([press_name, title, link])
df = pd.DataFrame(news_list, columns = ['Press', 'Title', 'Link'])

df.head()

언론사별 뉴스 Crawling: 100%|██████████| 82/82 [01:40<00:00,  1.22s/it]


,Press,Title,Link
0,세계일보,"檢, ‘제3자 뇌물 혐의’ 이재명 기소… 李 ""검찰 창작 수준 갈수록 떨어져""",https://n.news.naver.com/article/022/000394111...
1,세계일보,"“나라 구하다 죽었냐” 이태원 참사 막말 김미나, 의원직 유지할까?",https://n.news.naver.com/article/022/000394104...
2,세계일보,[단독] 미국 출시도 안된 제품이 ‘미국 1위’? 뷰즈 전자담배 광고에 소비자 혼란,https://n.news.naver.com/article/022/000394111...
3,세계일보,한국 골프장 카트가 세계서 가장 비싼 렌터카? [수민이가 화났어요],https://n.news.naver.com/article/022/000394114...
4,세계일보,김건희 여사 “진돗개 닮았다” 말에…국견 선물한 투르크 최고지도자,https://n.news.naver.com/article/022/000394100...


In [8]:
cwd = os.getcwd()
print(cwd)

/Users/gwangyeong/Desktop/Development_231120/과제/[240530~240614_Main_Project_05_Python,Swift] HaruSijack_Application/Data 분석/Scrapy Crawling


In [9]:
# 오늘 날짜 가져오기
today = datetime.now().strftime('%Y%m%d')

df.to_csv(f"Data/NAVER_News_List_{today}.csv")

In [10]:
# # 1. 언론사 목록 가져오기
# press_list = list(df.Press.unique())  # random.sample()은 Sequence Type의 Data만 지원하기 때문에 df.Press.unique()의 결과를 List로 변환해야 한다.

# # 2. 랜덤으로 10개의 언론사 선택
# selected_press = random.sample(press_list, 10)

# # # 3. 선택된 언론사에서 Random으로 1~5개의 Link 추출
# links = []
# for press in selected_press :
#   links.extend(df[df['Press'] == press].sample(random.randint(1, 5))['Link'].tolist())

# links

---

## 본문 수집하기

In [11]:
# # 기본 File Name 설정
# base_filename = df[df['Link'] == link]['Title'].values[0]
# extension = ".png"
# name_index = 1  # File Name에 추가될 숫자
# new_filename = base_filename + extension

# # 기본 Path 설정
# ## 오늘 날짜 가져오기
# today = datetime.now().strftime('%Y%m%d')
# base_path = f"Data/ScreenShot/By_Press/{today}"
# # By_Press 폴더 생성
# os.makedirs(base_path, exist_ok = True)
# path_index = 1  # Path에 추가될 숫자

# # 동일한 By_Press/today 폴더가 존재하는지 확인하고, 존재한다면 하위에 새로운 Folder 생성
# while Path(base_path).exists() :
#     file_path = base_path + f"{path_index}"
#     path_index += 1

# # File Name
# screenshot_Name = file_path + new_filename

# # 동일한 File Name이 존재하는지 확인하고, 존재한다면 새로운 File Name 생성
# while Path(new_filename).exists() :
#     new_filename = f"{base_filename}_{name_index}{extension}"
#     name_index += 1

In [12]:
# Chrome Browser와 Chrome Driver Version 확인 및 WebDriver 객체 생성
chrome_options = webdriver.ChromeOptions()
# ******************************************************
chrome_options.add_argument('headless') # Run chrome browser in the background
chrome_options.add_argument('window-size = 1920x1080')  # Chrome Browser Window Size
# ******************************************************
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

# 1. 언론사 목록 가져오기
press_list = list(df.Press.unique())  # random.sample()은 Sequence Type의 Data만 지원하기 때문에 df.Press.unique()의 결과를 List로 변환해야 한다.

# 2. 랜덤으로 10개의 언론사 선택
selected_press = random.sample(press_list, 10)

# # 3. 선택된 언론사에서 Random으로 1~5개의 Link 추출
links = []
for press in selected_press :
    links.extend(df[df['Press'] == press].sample(random.randint(1, 5))['Link'].tolist())

# tqdm Module을 이용하여 진행 상황 표시
pbar = tqdm(links, desc = "Crawling 진행 중", unit = "Link")

# 뉴스 기사 Link를 순회하며 기사 내용 추출
articles = []
for link in pbar :
    driver.get(link)

    # **************************************************************************
    # 기본 Path 설정
    today = datetime.now().strftime('%Y%m%d')   # 오늘 날짜 가져오기
    base_path = f"Data/ScreenShot/By_Press/{today}"

    ## By_Press 폴더 생성
    os.makedirs(base_path, exist_ok = True)

    # 동일한 By_Press/today Folder가 존재하는지 확인하고, 존재한다면 하위에 새로운 Folder 생성
    new_path = base_path
    while os.path.exists(new_path) :
        new_path = os.path.join(base_path, str(len(os.listdir(base_path)) + 1)) # index는 폴더 내 파일 개수 + 1로 설정
    
    # 새로운 Folder 생성
    os.makedirs(new_path, exist_ok = True)

    # 기본 File Name 설정
    base_filename = df[df['Link'] == link]['Title'].values[0]
    extension = ".png"
    name_index = 1  # File Name에 추가될 숫자
    new_filename = base_filename + extension

    # File Path 생성
    screenshot_name = os.path.join(new_path, new_filename)

    # 동일한 File Name이 존재하는지 확인하고, 존재한다면 새로운 File Name 생성
    while os.path.exists(screenshot_name) :
        new_filename = f"{base_filename}_{name_index}{extension}"
        screenshot_name = os.path.join(new_path, new_filename)
        name_index += 1

    # ScreenShot 촬영 전에 시간 두기. (Loading이 느릴수도 있으므로...)
    time.sleep(3)
    
    # Browser 최대화
    driver.maximize_window()
    
    # 현재 화면 Capture하기
    driver.save_screenshot(screenshot_name)
    # **************************************************************************

    html = driver.page_source
    article_soup = BeautifulSoup(html, "html.parser")
    content = article_soup.select_one("#contents")
    if content :
        # 공백과 HTML Tag 제거
        text = ' '.join(content.text.split())
        articles.append(text)
    # 요청 후 임의의 시간만큼 대기 (Page Loaded)
    time.sleep(random.uniform(0.5, 2.0))

# Browser 종료 (모든 Tab 종료)
driver.quit()

# 빈 문서가 있는지 확인
articles = [article for article in articles if article.strip()]

# 기사 내용이 제대로 수집되었는지 확인
for i, article in enumerate(articles) :
    print(f"기사 {i + 1}: {article[:100]}...")  # 기사 내용 앞부분만 출력

# NLTK를 이용하여 불용어 제거, 단어 토큰화, 표제어 추출
## 한국어 불용어 모음집 불러오기
stopword_list = pd.read_csv("Data/stopword.txt", header = None)
stopword_list[0] = stopword_list[0].apply(lambda x: x.strip())
korean_stopwords = stopword_list[0].to_numpy()

nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
tokenized_articles = []
for article in tqdm(articles, desc = "Text 처리 중", unit = "기사") :
    tokens = word_tokenize(article)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalnum() and word.lower() not in korean_stopwords]
    tokenized_articles.append(' '.join(tokens))

# 빈 문서가 있는지 다시 확인
tokenized_articles = [article for article in tokenized_articles if article.strip()]

# 각 기사별 Keyword 추출 (빈도 높은 단어)
keywords = []
for article in tokenized_articles :
    word_counts = Counter(article.split())
    common_words = word_counts.most_common(10)  # 상위 10개 단어 추출
    keywords.append([word for word, freq in common_words])

# Keyword 확인
for i, kw in enumerate(keywords) :
    print(f"기사 {i + 1} 키워드: {kw}")

Crawling 진행 중: 100%|██████████| 30/30 [02:22<00:00,  4.76s/Link]


기사 1: 연세의대 교수 비대위 12일 결의전의교협도 총회서 집단행동 논의 서울대병원 교수들이 오는 17일부터 무기한 전체휴진을 예고한 가운데 12일 오전 서울 종로구 서울대학교병원에서 한 ...
기사 2: 연합뉴스[서울경제] 12일 강릉 아침 최저기온이 26.3도를 기록해 전날 올해 전국 첫 열대야에 이어 이틀 연속 열대야가 발생해 후텁지근하고 견디기 힘든 무더운 밤이 이어졌다.열대...
기사 3: 활동적인 푸바오를 위한 맞춤형 공간가장 자연친화적인 선수핑기지서 생활샤오치지·셩이 대비 방사장 크기 작아[서울경제] 중국 판다보호연구센터 워룽선수핑기지 관계자들이 11일 공개한 푸...
기사 4: 사진=이미지투데이[서울경제] 세계적으로 통풍 환자의 수가 가파른 증가 추세를 보이고 있는 가운데 환자 연령대도 낮아지고 있다. 우리나라도 예외는 아니다.건강보험심사평가원의 국민관심...
기사 5: Video will play after Ad Next subject author Cancel "진실 밝히고 박정훈 명예회복"‥아들 순직 1년 만에 '첫 입장' MBC뉴스 Play ...
기사 6: Video will play after Ad Next subject author Cancel 헬스장에 '아줌마 출입 금지' 안내문‥이유는? MBC뉴스 Play 9.7K 0:00:0...
기사 7: [Food Trend] 건강에 좋은 60가지 유기산 담긴 흑초·누룩 소금 인기 초루에서 판매 중인 자연 발효 흑초. [초루 제공]김치, 된장, 식초, 요구르트의 공통점은 모두 ‘발...
기사 8: 시진핑 정권에 좌절한 30, 40대 中 중산층이 불법 이민 감행 지난해 10월 24일(현지 시간) 중국인들이 망명을 위해 멕시코 국경을 넘은 후 미국 캘리포니아주 자쿰바 인근에서 ...
기사 9: 기업 기증 새 물품을 시중 가격의 3분의 1에 판매… 수익금은 취약계층 일자리 제공5월 30일 오후 기자가 찾은 서울 중구 기증품 판매점 ‘굿윌스토어’는 평일인데도 사람들로 북적거...
기사 10: 최근 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gwangyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gwangyeong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Text 처리 중: 100%|██████████| 30/30 [00:01<00:00, 24.23기사/s]

기사 1 키워드: ['수', '구독', '닫기', '교수', '무기한', '휴진을', '응답은', '휴진', '있습니다', '언론사']
기사 2 키워드: ['구독', '닫기', '수', '기자', '있습니다', '언론사', '서울경제', '시원한', '기사를', '볼']
기사 3 키워드: ['수', '푸바오의', '푸바오가', '구독', '푸바오는', '닫기', '실외', '볼', '방사장', '지낼']
기사 4 키워드: ['수', '구독', '통풍', '닫기', '많이', '기자', '있습니다', '언론사', '서울경제', '있는']
기사 5 키워드: ['며', '구독', '수', '박정훈', 'subtitle', '아들의', '고', '있습니다', '해병대', '어머니는']
기사 6 키워드: ['subtitle', '수', '있습니다', '언론사', 'play', '480p', '한', '노', '기사를', '닫기']
기사 7 키워드: ['자연', '발효', '수', '건강에', '소금', '0', '주간동아', '있습니다', '언론사', '과정을']
기사 8 키워드: ['미국', '불법', '미국으로', '중국인', '중국', '지난해', '중국인은', '최근', '망명', '파나마']
기사 9 키워드: ['기증품', '수', '판매점', '새', '굿윌스토어', '는', '있는', '고', '있습니다', '한']
기사 10 키워드: ['금', '구리', '천연가스', '가격', '가격은', '올해', '최근', '약', '이후', '미국']
기사 11 키워드: ['수', '구독', '강형욱', '고발인은', '사내', '메신저', '법적', '것으로', '고', '없다']
기사 12 키워드: ['수', '구독', '심박', '닫기', '사자의', '애플워치', '기자', '있습니다', '언론사', '애플워치를']
기사 13 키워드: ['김정은', '6월', '오물', '수', '교란', '5월', '북한이', 'gps', '북한은', '북한']
기사 1

> base_path 설정 부분: <br>
os.path.exists(file_path) 조건을 사용하여 실제 경로가 존재하는지 확인합니다. <br>
경로가 존재하면 file_path에 새로운 폴더 번호를 추가하여 경로를 생성합니다.

> new_filename 생성 부분: <br>
os.path.exists(screenshot_name) 조건을 사용하여 실제 파일 경로가 존재하는지 확인합니다. <br>
파일이 존재하면 new_filename에 새로운 번호를 추가하여 파일 이름을 생성합니다.

> os.path.join() 함수는 운영 체제에 맞는 경로 구분자(Windows의 '', Unix/Linux의 '/')를 자동으로 처리해주기 때문에, <br>
상대경로나 절대경로를 모두 안전하게 연결할 수 있습니다.

---

In [13]:
# # Chrome Browser와 Chrome Driver Version 확인
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('headless')  # Run chrome browser in the background
# chrome_options.add_argument('window-size=1920x1080')  # Chrome Browser Window Size

# """
#     본문 수집 함수 작성:
#     - 주어진 Link에서 기사 본문을 수집하고 ScreenShot을 저장합니다.
#     - ScreenShot 저장 경로 설정, File Name 중복 처리 등 추가 기능을 포함합니다.
# """
# def fetch_article_content (link) :
#     """
#     주어진 URL에서 기사의 본문을 수집하는 함수
#     """
#     try :
#         driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)
#         driver.get(link)

#         # ************************************************************************************************
#         # 기본 Path 설정
#         today = datetime.now().strftime('%Y%m%d')
#         base_path = f"../Data/ScreenShot/By_Press/{today}"
#         os.makedirs(base_path, exist_ok = True)

#         # 동일한 By_Press/today Folder가 존재하는지 확인하고, 존재한다면 하위에 새로운 Folder 생성
#         new_path = base_path
#         while os.path.exists(new_path) :
#             new_path = os.path.join(base_path, str(len(os.listdir(base_path)) + 1)) # index는 Folder 내 File 갯수 + 1로 설정

#         # 새로운 Folder 생성
#         os.makedirs(new_path, exist_ok = True)

#         # 기본 File Name 설정
#         base_filename = df[df['Link'] == link]['Title'].values[0]
#         extension = ".png"
#         name_index = 1
#         new_filename = base_filename + extension

#         # File Path 생성
#         screenshot_name = os.path.join(new_path, new_filename)

#         # 동일한 File Name이 존재하는지 확인하고, 존재한다면 새로운 File Name 생성
#         while os.path.exists(screenshot_name) :
#             new_filename = f"{base_filename}_{name_index}{extension}"
#             screenshot_name = os.path.join(new_path, new_filename)
#             name_index += 1

#         # ScreenShot 촬영 전에 시간 두기. (Loading이 느릴수도 있으므로...)
#         time.sleep(3)
        
#         # Browser 최대화
#         driver.maximize_window()
        
#         # 현재 화면 Capture하기
#         driver.save_screenshot(screenshot_name)
#         # ************************************************************************************************

#         html = driver.page_source
#         driver.quit()  # Browser 종료 (모든 Tab 종료)
#         article_soup = BeautifulSoup(html, "html.parser")
#         content = article_soup.select_one("#contents")
#         if content :
#             # 공백과 HTML Tag 제거
#             text = ' '.join(content.text.split())
#             return text
#         else :
#             return None
        
#     except Exception as e :
#         print(f"Error fetching {link}: {e}")
#         return None


# """
#     Multiprocessing을 사용하여 본문 수집:
#     - 모든 Link를 가져와 List로 변환합니다.
#     - Pool(processes=4): 4개의 Process를 가진 Pool을 생성합니다.
#     - pool.imap(fetch_article_content, links): Link List를 동시에 처리합니다.
# """
# # 모든 Link 가져오기
# links = df['Link'].tolist()

# # Multi Processing을 사용하여 본문 수집
# pool = Pool(processes = 4)  # 4개의 Process를 가진 Pool을 생성
# results = list(
#     tqdm(
#         pool.imap(fetch_article_content, links),
#         total = len(links),
#         desc = "Crawling 진행 중",
#         unit = "Link"
#         )
# )
# pool.close()  # 작업이 끝난 후 Pool을 닫음
# pool.join()  # 모든 Process가 끝날 때까지 기다림

# # 빈 문서가 있는지 확인
# articles = [article for article in results if article and article.strip()]

# # 기사 내용이 제대로 수집되었는지 확인
# for i, article in enumerate(articles) :
#     print(f"기사 {i + 1}: {article[:100]}...")  # 기사 내용 앞부분만 출력

# # NLTK를 이용하여 불용어 제거, 단어 토큰화, 표제어 추출
# stopword_list = pd.read_csv("../Data/stopword.txt", header = None)
# stopword_list[0] = stopword_list[0].apply(lambda x: x.strip())
# korean_stopwords = stopword_list[0].to_numpy()

# nltk.download('punkt')
# nltk.download('wordnet')
# lemmatizer = WordNetLemmatizer()
# tokenized_articles = []
# for article in tqdm(articles, desc = "Text 처리 중", unit = "기사"):
#     tokens = word_tokenize(article)
#     tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalnum() and word.lower() not in korean_stopwords]
#     tokenized_articles.append(' '.join(tokens))

# # 빈 문서가 있는지 다시 확인
# tokenized_articles = [article for article in tokenized_articles if article.strip()]

# # 각 기사별 Keyword 추출 (빈도 높은 단어)
# keywords = []
# for article in tokenized_articles :
#     word_counts = Counter(article.split())
#     common_words = word_counts.most_common(10)  # 상위 10개 단어 추출
#     keywords.append([word for word, freq in common_words])

# # Keyword 확인
# for i, kw in enumerate(keywords) :
#     print(f"기사 {i + 1} 키워드: {kw}")